Wenn POS:
    Finde längsten Substring aus TK-Vokabular (Stämme)
    Präfix:
        Rekursiv längste Substrings bis komplett segmentiert
    Suffix:
        Rekursiv längste Substrings bis komplett segmentiert
    Rest:
        Aus atomarem Vokabular

LOAD FUNCTIONAL / LEXEMIC VOCAB

In [1]:
import collections as cl
from more_itertools import subslices

In [2]:
with open("../new_tokenizer/lex_vocab_raw.txt", encoding="utf8", mode="r") as lv:
    lm_raw = lv.read().split("\n")
    lv.close()

with open("../new_tokenizer/fun_vocab_raw.txt", encoding="utf8", mode="r") as fv:
    fm_raw = fv.read().split("\n")
    fv.close()

156520

INITIALIZE HASHABLES WITH RELATIVE WEIGHTS FOR EACH MORPHEME

In [3]:
fm_clean = [i for i in fm_raw if i != ""]  # select non-empty morphemes
fm_ncount = cl.Counter(fm_clean).most_common()
n_o_fm = len(fm_clean)
fm_rel = {k: v / n_o_fm if len(k) > 1 else 0 for k, v in fm_ncount}  # unary morphemes get no weight

lm_clean = [i for i in lm_raw if len(i) > 1]  # select only morphemes longer than 1 character
lm_ncount = cl.Counter(lm_clean).most_common()
n_o_lm = len(lm_clean)
lm_rel = {k: v / n_o_lm if len(k) > 1 else 0 for k, v in lm_ncount}  # unary morphemes get no weight
lm_rel

{'schrei': 0.0003197953309881676,
 'schnei': 0.0002942117045091142,
 'ei': 0.00028141989126958747,
 'au': 0.00028141989126958747,
 'rei': 0.00023025263831148065,
 'schie': 0.00023025263831148065,
 'lie': 0.00019187719859290055,
 'lau': 0.00019187719859290055,
 'wuch': 0.00019187719859290055,
 'wach': 0.00017908538535337383,
 'schla': 0.00017908538535337383,
 'ar': 0.00017908538535337383,
 'rau': 0.00017908538535337383,
 'wir': 0.00016629357211384713,
 'schau': 0.00016629357211384713,
 'stür': 0.00016629357211384713,
 'al': 0.00016629357211384713,
 'ie': 0.00016629357211384713,
 'grei': 0.00015350175887432043,
 'hau': 0.00015350175887432043,
 'schlie': 0.00015350175887432043,
 'blie': 0.00015350175887432043,
 'strei': 0.00015350175887432043,
 'blei': 0.00015350175887432043,
 'brau': 0.00015350175887432043,
 'in': 0.00015350175887432043,
 'wand': 0.00014070994563479373,
 'frühstück': 0.00014070994563479373,
 'flie': 0.00014070994563479373,
 'bekomm': 0.00014070994563479373,
 'schlaf': 0.

In [23]:
token = "überschreiben"

In [54]:
[0,lm_rel["ge"],lm_rel["geh"],lm_rel["gehe"]]

[0, 2.5583626479053406e-05, 6.395906619763352e-05, 1.2791813239526703e-05]

In [8]:
print((fm_rel["üb"]+ fm_rel["er"]+lm_rel["schreib"]+fm_rel["en"]) > (fm_rel["über"]+lm_rel["schreib"]+fm_rel["en"]))
print((lm_rel["geh"]+fm_rel["en"]) > (fm_rel["ge"]+0+fm_rel["en"]))

True
False


In [34]:
[i for i in fm_rel if i.startswith("ü")]

['über', 'üb', 'übe', 'ü']

In [35]:
[i for i in fm_rel if i.startswith("s")]

['st', 's', 'sch', 'ss', 'sc']

COMBINE BOTH LEX AND FUN

In [2]:
with open("../new_tokenizer/lex_vocab_raw.txt", encoding="utf8", mode="r") as lv:
    lm_raw = lv.read().split("\n")
    lv.close()

with open("../new_tokenizer/fun_vocab_raw.txt", encoding="utf8", mode="r") as fv:
    fm_raw = fv.read().split("\n")
    fv.close()

In [3]:
token = "beschreiben"

In [4]:
fm_raw_clean = [i for i in fm_raw if i != ""]

In [5]:
lm_raw_clean = [i for i in lm_raw if i != ""]

In [6]:
lmfm = lm_raw_clean+fm_raw_clean

In [7]:
lm_clean = [i for i in lmfm]
lm_ncount = cl.Counter(lm_clean).most_common()
n_o_lm = len(lm_clean)
lmfm = {k: v / n_o_lm if len(k) > 1 else 0 for k, v in lm_ncount}  # unary morphemes get no weight


In [8]:
len(lmfm)

43700

In [11]:
print((lmfm["üb"]+ lmfm["er"]+lmfm["schreib"]+lmfm["en"]) > (lmfm["über"]+lmfm["schreib"]+lmfm["en"]))
print((lmfm["geh"]+lmfm["en"]) > (lmfm["ge"]+0+lmfm["en"]))

True
False


In [9]:
tv = [i for i in lmfm if i in token]  # morpheme vocab for token (functional + lexical)

In [14]:
segmentation = ""
token = "beschreiben"
m1 = "be"
m1[0:] == token[0:len(m1)]

KeyboardInterrupt: 

if morphem startswith:
    merke kosten
    finde strip, return morphem + restword

In [23]:

class Segmenter:
    def __init__(self, token, vocab):
        self.token = token
        self.vocab = vocab

    subvocab = ["".join(i) for i in subslices(token) if "".join(i) in tv]


    def thefunction(self, token, subvocab,  start=0, stop=int, segments=None):
        if segments is None:
            segments = []
        if start<stop-1:
            return segments
        while start<stop-1:
            new_morpheme = [i for i in subvocab if token.startswith(i)]
            for m in new_morpheme:
                rest = token[len(m):]
                start+=len(m)
                segments.append(m)
                thefunction(token=rest, subvocab=subvocab, start=start, segments=segments, stop=stop)


In [134]:
tk = "beurteilen"
subvocab = [i for i in lmfm if i in tk]
segmentations = []

In [140]:

def segmenter(token, segments=None, start=0, stop=int):
    if segments is None:
        segments = []
    if start == stop:
        segmentations.append(segments)
    else:
        new_morpheme = [i for i in subvocab if token.startswith(i)]
        for m in new_morpheme:
            rest = token[len(m):]
            start += len(m)
            segments.append(m)
            segmenter(token=rest, segments=segments.copy(), start=start, stop=stop)
            segments = segments[:-1]
            start -= len(m)

In [143]:
import timeit


gfs = timeit.default_timer()

segs = segmenter(token=tk, stop=len(tk))

asd = timeit.default_timer()

print('Time: ', asd - gfs)


Time:  0.0010547400015639141


In [138]:
for i in segmentations:
    print(i)

['b', 'e', 'u', 'rt', 'e', 'i', 'l', 'e', 'n']
['b', 'e', 'u', 'rt', 'e', 'i', 'l', 'en']
['b', 'e', 'u', 'rt', 'e', 'i', 'le', 'n']
['b', 'e', 'u', 'rt', 'e', 'i', 'len']
['b', 'e', 'u', 'rt', 'e', 'il', 'e', 'n']
['b', 'e', 'u', 'rt', 'e', 'il', 'en']
['b', 'e', 'u', 'rt', 'ei', 'l', 'e', 'n']
['b', 'e', 'u', 'rt', 'ei', 'l', 'en']
['b', 'e', 'u', 'rt', 'ei', 'le', 'n']
['b', 'e', 'u', 'rt', 'ei', 'len']
['b', 'e', 'u', 'rt', 'eil', 'e', 'n']
['b', 'e', 'u', 'rt', 'eil', 'en']
['b', 'e', 'u', 'rt', 'eile', 'n']
['b', 'e', 'u', 'r', 't', 'e', 'i', 'l', 'e', 'n']
['b', 'e', 'u', 'r', 't', 'e', 'i', 'l', 'en']
['b', 'e', 'u', 'r', 't', 'e', 'i', 'le', 'n']
['b', 'e', 'u', 'r', 't', 'e', 'i', 'len']
['b', 'e', 'u', 'r', 't', 'e', 'il', 'e', 'n']
['b', 'e', 'u', 'r', 't', 'e', 'il', 'en']
['b', 'e', 'u', 'r', 't', 'ei', 'l', 'e', 'n']
['b', 'e', 'u', 'r', 't', 'ei', 'l', 'en']
['b', 'e', 'u', 'r', 't', 'ei', 'le', 'n']
['b', 'e', 'u', 'r', 't', 'ei', 'len']
['b', 'e', 'u', 'r', 't', 'eil'

In [160]:
ws = dict()  # weighted segmentations
tk = "beurteilen"
len_tk = len(tk)
for s in segmentations:
    # ws[sum([lmfm[w] for w in s])/len(s)] = s
    coverage = [len(i)/len_tk for i in s]
    rel_freq = [lmfm[i] for i in s]
    cover_rf = sum([i*j for i,j in zip(coverage, rel_freq)])
    ws[cover_rf] = s
ws[max(ws.keys())]

['be', 'ur', 'te', 'il', 'en']

1064